In [1]:
import torch
import numpy as np
from tqdm import tqdm

In [2]:

# Define alphas as torch tensors with requires_grad=True
alpha_1 = torch.tensor(1.013, dtype=torch.float64, requires_grad=True)
alpha_2 = torch.tensor(0.2119, dtype=torch.float64, requires_grad=True)
alpha_3 = torch.tensor(0.1406, dtype=torch.float64, requires_grad=True)
alpha_4 = torch.tensor(0.003, dtype=torch.float64, requires_grad=True)

alphas = [alpha_1, alpha_2, alpha_3, alpha_4]

# psi function using torch
def psi(x, y, alpha_1, alpha_2, alpha_3, alpha_4):
    r1 = torch.norm(x)
    r2 = torch.norm(y)
    r12 = torch.norm(x - y)

    term1 = torch.exp(-2 * (r1 + r2))
    term2 = 1 + 0.5 * r12 * torch.exp(-alpha_1 * r12)
    term3 = 1 + alpha_2 * (r1 + r2) * r12 + alpha_3 * (r1 - r2)**2 - alpha_4 * r12

    return term1 * term2 * term3

# Tensors with gradients enabled
x = torch.tensor([0.2, 0.3, 0.1], dtype=torch.float64, requires_grad=True)
y = torch.tensor([0.1, 0.4, 0.3], dtype=torch.float64, requires_grad=True)

# Compute psi
psi_val = psi(x, y, *alphas)

# First gradients w.r.t. x and y
grad_x = torch.autograd.grad(psi_val, x, create_graph=True)[0]
grad_y = torch.autograd.grad(psi_val, y, create_graph=True)[0]

# Laplacians
laplacian_x = sum(torch.autograd.grad(grad_x[i], x, retain_graph=True, create_graph=True)[0][i] for i in range(3))
laplacian_y = sum(torch.autograd.grad(grad_y[i], y, retain_graph=True, create_graph=True)[0][i] for i in range(3))

lap = laplacian_x + laplacian_y
# Derivatives of Laplacians w.r.t. alphas
dlap_dalpha = torch.autograd.grad(lap, alphas, retain_graph=True)


print(dlap_dalpha)

(tensor(-0.5851, dtype=torch.float64), tensor(2.5131, dtype=torch.float64), tensor(0.6771, dtype=torch.float64), tensor(-2.5998, dtype=torch.float64))


In [3]:
"""
def local_energy(psi_val, r1, r2):
    grad_x = torch.autograd.grad(psi_val, r1, create_graph=True)[0]
    grad_y = torch.autograd.grad(psi_val, r2, create_graph=True)[0]

    # Laplacians
    laplacian_x = sum(torch.autograd.grad(grad_x[i], r1, retain_graph=True, create_graph=True)[0][i] for i in range(3))
    laplacian_y = sum(torch.autograd.grad(grad_y[i], r2, retain_graph=True, create_graph=True)[0][i] for i in range(3))

    ke = - 0.5 * (laplacian_x + laplacian_y)

    return - 2 / torch.norm(r1) - 2 / torch.norm(r2) + 1 / torch.norm(r1 - r2) + ke
"""

def local_energy(psi_val, r1, r2):
    grad_r1 = torch.autograd.grad(psi_val, r1, create_graph=True)[0]
    grad_r2 = torch.autograd.grad(psi_val, r2, create_graph=True)[0]

    lap_r1 = torch.autograd.grad(grad_r1, r1, grad_outputs=torch.ones_like(grad_r1), create_graph=True)[0]
    lap_r2 = torch.autograd.grad(grad_r2, r2, grad_outputs=torch.ones_like(grad_r2), create_graph=True)[0]

    laplacian = lap_r1.sum() + lap_r2.sum()
    ke = -0.5 * laplacian

    potential = -2 / torch.norm(r1) - 2 / torch.norm(r2) + 1 / torch.norm(r1 - r2)

    return ke + potential


In [4]:
L = 1
r1 = torch.rand(3, requires_grad=True) * 2 * L - L
r2 = torch.rand(3, requires_grad=True) * 2 * L - L #random number from -L to L
E = 0
E2 = 0
Eln_average = 0
ln_average = 0
rejection_ratio = 0
step = 0
max_steps = 500
N = 10000
dlap_dalpha = 0
xs = []
ys = []
psi_vals = []

for i in tqdm(range(N)):
    flag = False
    chose = np.random.rand()
    step = step + 1
    if chose < 0.5:
        r1_trial = r1 + 0.5 * (torch.rand(3) * 2 * L-L)
        r2_trial = r2
    else:
        r2_trial = r2 + 0.5 * (torch.rand(3) * 2 * L-L)
        r1_trial = r1


    psi_val = psi(r1, r2, *alphas)
    psi_val_trial = psi(r1_trial, r2_trial, *alphas)
    

    if psi_val_trial.item() >= psi_val.item():
        r1 = r1_trial
        r2 = r2_trial
        flag = True
    
    else:
        dummy = np.random.rand()
        if dummy < psi_val_trial.item() / psi_val.item():
            r1 = r1_trial
            r2 = r2_trial
        else:
            rejection_ratio += 1./N
            
    if step > max_steps:
        xs.append(r1)
        ys.append(r2)

        if flag:
            psi_vals.append(psi_val_trial)
        else:
            psi_vals.append(psi_val)
        """
        local_E = local_energy(psi_val_trial, r1, r2)
        E += local_E / (N - max_steps)
        E2 += local_E ** 2 / (N - max_steps)
        dlap_dalpha += torch.tensor(torch.autograd.grad(local_E, alphas, retain_graph=True)) / (N - max_steps)
        """





  0%|          | 0/10000 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [00:04<00:00, 2157.45it/s]


In [5]:
alpha_1 = torch.tensor(1.013, dtype=torch.float64, requires_grad=True)
alpha_2 = torch.tensor(0.2119, dtype=torch.float64, requires_grad=True)
alpha_3 = torch.tensor(0.1406, dtype=torch.float64, requires_grad=True)
alpha_4 = torch.tensor(0.003, dtype=torch.float64, requires_grad=True)

In [6]:
from torch.func import grad, vmap

In [7]:
import torch
from torch.autograd.functional import jacobian

# Original scalar-valued psi
def psi(X):
    x = X[:3]
    y = X[3:6]
    alpha_1, alpha_2, alpha_3, alpha_4 = X[6:]
    r1 = torch.norm(x)
    r2 = torch.norm(y)
    r12 = torch.norm(x - y)

    term1 = torch.exp(-2 * (r1 + r2))
    term2 = 1 + 0.5 * r12 * torch.exp(-alpha_1 * r12)
    term3 = 1 + alpha_2 * (r1 + r2) * r12 + alpha_3 * (r1 - r2)**2 - alpha_4 * r12

    return term1 * term2 * term3

In [171]:
t = vmap(psi)

In [8]:
inputs = []
for i in range(len(xs)):
    inputs.append(torch.tensor([*xs[i], *ys[i], alpha_1, alpha_2, alpha_3, alpha_4]))

In [9]:
inputs = torch.stack(inputs)

In [10]:
s = vmap(grad(psi))

## Verifying the analytical expressions for the deriviatives

In [11]:
from torch.autograd.functional import jacobian, hessian

In [86]:
def psi__(X):
    x = X[:3]
    y = X[3:6]
    alpha_1, alpha_2, alpha_3, alpha_4 = X[6:]
    r1 = torch.norm(x)
    r2 = torch.norm(y)
    r12 = torch.norm(x - y)

    term1 = torch.exp(-2 * (r1 + r2))
    term2 = 1 + 0.5 * r12 * torch.exp(-alpha_1 * r12)
    term3 = 1 + alpha_2 * (r1 + r2) * r12 + alpha_3 * (r1 - r2)**2 - alpha_4 * r12

    return term3
    #return #term1 * term2 * term3

In [ ]:
def psi_c_first(X):
    x = X[:3]
    y = X[3:6]
    alpha_1, alpha_2, alpha_3, alpha_4 = X[6:]
    r1 = torch.norm(x)
    r1_ = sum(x)
    r2_ = sum(y)
    r2 = torch.norm(y)
    r12 = torch.norm(x - y)
    r12_ = sum(x - y)

    i = 0

    return [alpha_2 * (r1 + r2) * ((x[i] - y[i])/r12) + alpha_2 * r12 * (x[i]/r1) + 2 * (r1 - r2) * alpha_3 * (x[i]/r1) - alpha_4 * (x[i] - y[i])/r12 for i in range(3)] + [- alpha_2 * (r1 + r2) * ((x[i] - y[i])/r12) + alpha_2 * r12 * (y[i]/r2) + 2 * (r1 - r2) * alpha_3 * (y[i]/r2) + alpha_4 * (x[i] - y[i])/r12 for i in range(3)]

In [93]:
t = jacobian(psi__, inputs[sample])

In [102]:
s = psi_c_first(inputs[sample])

In [95]:
print(t)

tensor([ 0.4480,  1.7283,  0.1417,  0.0583, -0.8553, -0.2798,  0.0000, 10.7326,
         2.2143, -2.5474], dtype=torch.float64)


In [103]:
print(s)

tensor(0.4480, dtype=torch.float64)


In [76]:
s

[tensor(-0.0002, dtype=torch.float64),
 tensor(-0.0004, dtype=torch.float64),
 tensor(3.0665e-05, dtype=torch.float64),
 tensor(-0.0004, dtype=torch.float64),
 tensor(-3.1355e-05, dtype=torch.float64),
 tensor(0.0003, dtype=torch.float64)]

In [52]:
from analytical_expressions import psi_a_second, psi_b_second, psi_c_second

In [60]:
sample = 133

hes = torch.diag(hessian(psi__, inputs[sample]))[0:6]
psia__ = psi_a_second(inputs[sample])
psib__ = psi_b_second(inputs[sample])
psic__ = psi_c_second(inputs[sample])

In [68]:
psia__

[tensor(3.3238e-05, dtype=torch.float64),
 tensor(0.0007, dtype=torch.float64),
 tensor(-0.0001, dtype=torch.float64),
 tensor(0.0005, dtype=torch.float64),
 tensor(-0.0003, dtype=torch.float64),
 tensor(8.8504e-05, dtype=torch.float64)]

In [66]:
analytical

[tensor(3.3238e-05, dtype=torch.float64),
 tensor(0.0007, dtype=torch.float64),
 tensor(-0.0001, dtype=torch.float64),
 tensor(0.0005, dtype=torch.float64),
 tensor(-0.0003, dtype=torch.float64),
 tensor(8.8504e-05, dtype=torch.float64),
 tensor(-0.0234, dtype=torch.float64),
 tensor(0.0197, dtype=torch.float64),
 tensor(-0.0210, dtype=torch.float64),
 tensor(-0.0234, dtype=torch.float64),
 tensor(0.0197, dtype=torch.float64),
 tensor(-0.0210, dtype=torch.float64),
 tensor(0.6828, dtype=torch.float64),
 tensor(0.6826, dtype=torch.float64),
 tensor(0.6590, dtype=torch.float64),
 tensor(0.5477, dtype=torch.float64),
 tensor(0.0805, dtype=torch.float64),
 tensor(0.5431, dtype=torch.float64)]

In [65]:
for i, j in zip(hes, analytical):
    print(i)
    print(j)
    print(i==j)

tensor(9.5743e-05, dtype=torch.float64)
tensor(3.3238e-05, dtype=torch.float64)
tensor(False)
tensor(0.0015, dtype=torch.float64)
tensor(0.0007, dtype=torch.float64)
tensor(False)
tensor(-0.0004, dtype=torch.float64)
tensor(-0.0001, dtype=torch.float64)
tensor(False)
tensor(0.0019, dtype=torch.float64)
tensor(0.0005, dtype=torch.float64)
tensor(False)
tensor(-0.0012, dtype=torch.float64)
tensor(-0.0003, dtype=torch.float64)
tensor(False)
tensor(0.0003, dtype=torch.float64)
tensor(8.8504e-05, dtype=torch.float64)
tensor(False)


In [42]:
print(psi_a_second(inputs[sample]))

[tensor(3.3238e-05, dtype=torch.float64), tensor(0.0007, dtype=torch.float64), tensor(-0.0001, dtype=torch.float64), tensor(0.0005, dtype=torch.float64), tensor(-0.0003, dtype=torch.float64), tensor(8.8504e-05, dtype=torch.float64)]


In [141]:
sample = 133

hes = hessian(psi_a, inputs[sample])
print(f"Hessian is {torch.diag(hes)[0:6]}")
print(f"Analytical is {psi_a_(inputs[sample])}")

Hessian is tensor([-0.0021,  0.0081, -0.0031,  0.0003, -0.0027,  0.0057],
       dtype=torch.float64)
Analytical is [tensor(-0.0021, dtype=torch.float64, grad_fn=<MulBackward0>), tensor(0.0081, dtype=torch.float64, grad_fn=<MulBackward0>), tensor(-0.0031, dtype=torch.float64, grad_fn=<MulBackward0>), tensor(0.0003, dtype=torch.float64, grad_fn=<MulBackward0>), tensor(-0.0027, dtype=torch.float64, grad_fn=<MulBackward0>), tensor(0.0057, dtype=torch.float64, grad_fn=<MulBackward0>)]


In [201]:
hes = hessian(psi_b, inputs[sample])
jac = jacobian(psi_b_first_, inputs[sample])

print(sum(torch.diag(hes)[:6]))
#print(sum(jac[3:6]))

tensor(-0.0786, dtype=torch.float64)


In [202]:
sum(jacobian(psi_b_first_, inputs[sample])[3:6]) + sum(jacobian(psi_b_first, inputs[sample])[:3])

tensor(-0.1673, dtype=torch.float64)